<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/mrkim21apps/Oxford3Kaudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio gtts

In [ ]:
!pip install pydub
!apt-get install ffmpeg

In [ ]:
import gradio as gr
from gtts import gTTS
import pandas as pd
from pydub import AudioSegment
import io
import numpy as np

# Read the CSV data directly for demonstration purposes
# Here you should load your actual data

csv_file = "/content/Oxford3K.csv"

# Read the CSV file
data = pd.read_csv(csv_file)

df = pd.DataFrame(data)


def generate_speech(x, y):
    # Ensure x and y are integers
    x, y = int(x), int(y)

    # Create an empty audio segment for padding
    combined_audio = AudioSegment.silent(duration=1000)  # Starting with 1 second of silence for padding

    # Generate speech for each entry in the specified range
    for index, row in df.iloc[x-1:y].iterrows():
        sentence = f"Number {row['SID']}. {row['WORD']} is a {row['POS']}."
        tts = gTTS(text=sentence, lang='en')
        mp3_fp = io.BytesIO()
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)
        sentence_audio = AudioSegment.from_file(mp3_fp, format='mp3')
        # Add the sentence audio and 2 seconds of silence
        combined_audio += sentence_audio + AudioSegment.silent(duration=2000)

    # Export the combined audio to a BytesIO object and return the bytes
    mp3_io = io.BytesIO()
    combined_audio.export(mp3_io, format='mp3')
    mp3_io.seek(0)
    return mp3_io.read()

# Define the Gradio interface
iface = gr.Interface(
    fn=generate_speech,
    inputs=[
        gr.Number(label="Start Sentence Number (x)"),  # Assume the `default` parameter is removed if it causes an error
        gr.Number(label="End Sentence Number (y)")
    ],
    outputs=gr.Audio(label="Generated Speech"),
    title="Speech Generator",
    description="Generate speech from the CSV data. Specify the start and end sentence numbers."
)


# Launch the Gradio app with sharing enabled

iface.launch(share=True, debug=True)

# By category (option)

In [ ]:
import gradio as gr
from gtts import gTTS
from pydub import AudioSegment
import pandas as pd
import io

# Load the DataFrame
csv_file = "/content/Oxford3K.csv"  # Update this path
data = pd.read_csv(csv_file)

def generate_speech(level, x, y):
    # Filter df based on selected level
    filtered_df = data[data['LEVEL'] == level]

    # Ensure x and y are integers and within range
    x, y = int(x), int(y)
    filtered_df = filtered_df[(filtered_df['SID'] >= x) & (filtered_df['SID'] <= y)]

    combined_audio = AudioSegment.silent(duration=1000)  # Start with silence for padding

    for _, row in filtered_df.iterrows():
        sentence = f"Number {row['SID']}. {row['WORD']} is {row['POS']}."
        tts = gTTS(text=sentence, lang='en')
        mp3_fp = io.BytesIO()
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)
        sentence_audio = AudioSegment.from_file(mp3_fp, format="mp3")
        combined_audio += sentence_audio + AudioSegment.silent(duration=2000)

    mp3_io = io.BytesIO()
    combined_audio.export(mp3_io, format='mp3')
    mp3_io.seek(0)
    return mp3_io.getvalue()

# Interface with level selection
iface = gr.Interface(
    fn=generate_speech,
    inputs=[
        gr.Dropdown(label="Select Level", choices=['B1', 'B2']),  # Adapt choices based on your data
        gr.Number(label="Start Sentence Number (x)"),
        gr.Number(label="End Sentence Number (y)")
    ],
    outputs=gr.Audio(label="Generated Speech"),
    title="Speech Generator",
    description="Select a level and specify the start and end sentence numbers within that level."
)

iface.launch(share=True, debug=True)
